In [18]:
import time
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

# Data Loading
We first define a function for downloading and loading MNIST. 

In [19]:
mnist = input_data.read_data_sets(train_dir = "./MNIST_data", one_hot=True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


#  Implementation of an Autoencoder

In [22]:
def predict(x):
    """predict returns prediction given input
    """
    
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    
    conv1 = tf.layers.conv2d(x_image, filters=8, kernel_size=(3,3), strides=(1,1), padding='SAME', use_bias=True, name='conv1')
    maxpool1 = tf.layers.max_pooling2d(conv1, pool_size=(2,2), strides=(2,2), name='pool1')

    conv2 = tf.layers.conv2d(maxpool1, filters=4, kernel_size=(3,3), strides=(1,1), padding='SAME', use_bias=True, name='conv2')
    maxpool2 = tf.layers.max_pooling2d(conv2, pool_size=(2,2), strides=(2,2), name='pool2')
    
    conv3 = tf.layers.conv2d(maxpool2, filters=2, kernel_size=(3,3), strides=(1,1), padding='SAME', use_bias=True, name='conv3')
    conv2d_transpose1 = tf.layers.conv2d_transpose(conv3, filters=4, kernel_size=(2, 2), padding='SAME', strides=2, name='conv2d_transpose1')

    conv4 = tf.layers.conv2d(conv2d_transpose1, filters=4, kernel_size=(3,3), strides=(1,1), padding='SAME', use_bias=True, name='conv4')
    conv2d_transpose2 = tf.layers.conv2d_transpose(conv4, filters=8, kernel_size=(2, 2), padding='SAME', strides=2, name='conv2d_transpose2')
    
    conv5 = tf.layers.conv2d(conv2d_transpose2, filters=8, kernel_size=(3,3), strides=(1,1), padding='SAME', use_bias=True, name='conv5')
    output_conv = tf.layers.conv2d_transpose(conv5, filters=1, kernel_size=(1, 1), padding='SAME', strides=1, name='output_conv')
    
    return output_conv

    
def train(learning_rate=0.01, max_epochs=10, 
              batch_size=64):
        """ Train network on the given data. """
        
        # Define placeholder for x
        x = tf.placeholder(tf.float32, [None, 784])

        # Define placeholder for y
        y_ = tf.placeholder(tf.float32, [None, 10])

        # Predict given the data
        y_conv = predict(x)
        
        #loss
        loss = tf.reduce_mean(y_conv - y_)
    
        # Define loss and optimizer
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

        correct_prediction = tf.equal(y_conv, y_)
        correct_prediction = tf.cast(correct_prediction, tf.float32)
        accuracy = tf.reduce_mean(correct_prediction)
        
        train_acc = list()
        val_acc = list()
        test_acc = list()
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            
            for epoch in range(max_epochs):
                for _ in range(int(mnist.train.num_examples/batch_size)):
                    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
                    train_step.run(feed_dict={x: batch_xs, y_: batch_ys})
                
                print('Epoch', epoch, 'completed out of', max_epochs)
                
                train_accuracy = accuracy.eval(feed_dict={
                        x: mnist.train.images, y_: mnist.train.labels})
                
                validation_accuracy = accuracy.eval(feed_dict={
                    x: mnist.eval.images, y_: mnist.eval.labels})
                
                test_accuracy = accuracy.eval(feed_dict={
                    x: mnist.test.images, y_: mnist.test.labels})
                
                train_acc.append(train_accuracy)
                val_acc.append(validation_accuracy)
                test_acc.append(test_accuracy)
                
                print('epoch %d, training accuracy %g' % (epoch, train_accuracy))
                print('epoch %d, validation accuracy %g' % (epoch, train_accuracy))
                print('epoch %d, test accuracy %g' % (epoch, train_accuracy))
                
            return (train_acc, val_acc, test_acc)

# Training on MNIST
Finally we can let our network run on the MNIST dataset!

In [23]:
train()

ValueError: Variable conv1/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-13-a0abf7e757d9>", line 7, in predict
    conv1 = tf.layers.conv2d(x_image, filters=8, kernel_size=(3,3), strides=(1,1), padding='SAME', use_bias=True, name='conv1')
  File "<ipython-input-13-a0abf7e757d9>", line 65, in train
    y_conv = predict(x)
  File "<ipython-input-14-17783c99efbe>", line 4, in <module>
    validation_accuracy.append(train(learning_rate=lr)[1])


# Task 2: Changing the Learning Rate

In [21]:
learning_rates = [0.1, 0.01, 0.001]
validation_accuracy = list()
for lr in learning_rates:
    validation_accuracy.append(train(learning_rate=lr)[1])

for lr, va in zip(learning_rates, validation_accuracy):
    plt.plot(range(1,len(va)), va, label='Learning rate:%s' % lr)
    plt.xlabel('number of epochs')
    plt.ylabel('validation accuracy')
plt.show()

ValueError: Variable conv1/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-13-a0abf7e757d9>", line 7, in predict
    conv1 = tf.layers.conv2d(x_image, filters=8, kernel_size=(3,3), strides=(1,1), padding='SAME', use_bias=True, name='conv1')
  File "<ipython-input-13-a0abf7e757d9>", line 65, in train
    y_conv = predict(x)
  File "<ipython-input-14-17783c99efbe>", line 4, in <module>
    validation_accuracy.append(train(learning_rate=lr)[1])


# Task 3: Run Time

In [2]:
filters = [8, 16, 32, 64, 128, 256]
times = list()
parameters_count = list()
for filter_number in filters:
    t0 = time.time()
    parameters_count.append(parameter_number(filter_number))
    train(filter = filter_number)
    t1 = time.time()
    times.append(t0, t1)

Plotting parameters vs runtime

In [ ]:
plt.plot(parameters_count, times)
plt.xlabel('number of parameters')
plt.ylabel('runtime')
plt.show()